In [134]:
from __future__ import division
%matplotlib inline
import numpy as np
import pickle
import pymysql
import os
import random
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
from nltk import ngrams
import gensim
from gensim.models.word2vec import Word2Vec
import nltk.tokenize
import nltk.data
from nltk.tokenize import word_tokenize

In [135]:
# import pymysql
# db = pymysql.connect(host="localhost", port=3306, user="vicky", passwd="vicky9790851962", db="fulltextdarts", charset="utf8")
# cursor = db.cursor()
# query="SELECT DISTINCT filename, textdata FROM fulltextdarts.plaintext pt JOIN darts.decision d ON pt.decision_fk=d.id JOIN darts.court c ON court_fk=c.id JOIN darts.docket dk ON d.docket_fk=dk.id JOIN judges.euipo_docket_lawyer euipodl ON euipodl.opp_number=dk.reference WHERE c.id=1121 AND pt.language='en' AND dk.reference REGEXP '[0-9]{9}' ORDER BY RAND(123);"
# cursor.execute(query)


# data=pd.DataFrame(columns=["filename","text_data"])
# i=0
# for row in cursor:
#     data.loc[i]=row;i+=1

# judges_df=pd.read_excel(open("tf_training_set.xlsx","rb"),parse_cols="B,C,D")

# temp_df=pd.DataFrame(columns=["judge_name_1","judge_name_2","judge_name_3"])

# frames=[data,temp_df]
# full_data=pd.concat(frames)

# full_data.loc[0:len(judges_df),["judge_name_1","judge_name_2","judge_name_3"]]=judges_df.loc[0:len(judges_df),["judge_name_1","judge_name_2","judge_name_3"]]

In [136]:
# sentences=[full_data['text_data'][index].lower() for index in range(len(full_data['text_data']))]
# model=Word2Vec([word_tokenize(s) for s in sentences], size=150, min_count=1, window=5,  workers=12)

In [137]:
# pickle.dump(full_data,open("full_data.p","wb"))
# pickle.dump(model,open("model.p","wb"))

In [5]:
full_data=pickle.load(open("full_data.p","rb"))
model=pickle.load(open("model.p","rb"))

In [6]:
# for index in range(70):
#     print index
#     count=0
#     for judge_name in [full_data.judge_name_1[index],full_data.judge_name_2[index],full_data.judge_name_3[index]]:
#         tokenized_judge_name= word_tokenize(judge_name)

#         print "B-JN is "+tokenized_judge_name[0]+" "+str(count+1)

#         try:
#             for word in tokenized_judge_name[1:-1]:
#                 print "I-JN is "+word+" "+str(count+1)
#         except:
#             pass

#         print "E-JN is "+tokenized_judge_name[-1]+" "+str(count+1)
#         print "\n"
#         count+=1

In [7]:
def shape_w2v_gram_features_extraction(text,context_len,word2vec_length=150):
    tokens=word_tokenize(text)
    tokens_lower=[word.lower() for word in tokens]

    tokens_SS=["DOCSTART"]*context_len+word_tokenize(text)+["DOCSTOP"]*context_len
    tokens_SS_lower=[word.lower() for word in tokens_SS]
    tokens_SS_shape_features=[[int(unicode(word).isnumeric()),int(unicode(word).isalpha()),int(unicode(word).islower()),int(unicode(word).isupper())] for word in tokens_SS]

    word2vec_features=[]
    for word in tokens_lower:
        try:
            word2vec_features.append(list(model[word]))
        except:
            word2vec_features.append(list(np.random.rand(word2vec_length,1)))
    
    word2vec_SS_features=[list(np.zeros_like((word2vec_features[0])))]*context_len+word2vec_features+[list(np.zeros_like((word2vec_features[0])))]*context_len


    tokens_SS_grams = list(ngrams(tokens_SS, context_len*2+1))
    tokens_SS_shape_features_grams = np.asarray(list(ngrams(tokens_SS_shape_features, context_len*2+1)))
    word2vec_SS_features_grams=np.asarray(list(ngrams(word2vec_SS_features, context_len*2+1)))


    check1=tokens_SS_shape_features_grams.reshape(len(tokens_SS_grams),-1)
    check2=word2vec_SS_features_grams.reshape(len(tokens_SS_grams),-1)
    shape_w2v_gram_features=np.concatenate((check1,check2),axis=1)
    return tokens,shape_w2v_gram_features

In [8]:
def tf_training_set_labels_extraction(text,judge_names):
    tokens=word_tokenize(text)
    y_label=np.zeros(len(tokens))

    if judge_names[1]==" ":
            tokenized_judge_name= word_tokenize(judge_names[0])
            print tokenized_judge_name
            y_label[tokens.index(tokenized_judge_name[0])]=1

            try:
                for word in tokenized_judge_name[1:-1]:
                    y_label[tokens.index(word)]=2
            except:
                pass
            y_label[tokens.index(tokenized_judge_name[-1])]=3

    else:
        count=0
        for judge_name in judge_names:
            tokenized_judge_name= word_tokenize(judge_name)

            y_label[tokens.index(tokenized_judge_name[0])]=count+1

            try:
                for word in tokenized_judge_name[1:-1]:
                    y_label[tokens.index(word)]=count+1
            except:
                pass
            y_label[tokens.index(tokenized_judge_name[-1])]=count+1
            count+=1
    return tokens,y_label

In [9]:
def tf_test_set_labels_extraction(text):
    tokens=word_tokenize(text)
    y_label=np.zeros(len(tokens))
    return tokens,y_label

In [70]:
data_tokens=[]
data_features=[]
data_labels=[]
data_filenames=[]

full_data=full_data.fillna(" ")

context_len=3
for index in range(200):

    text=full_data.loc[index,"text_data"]
    judge_names=[full_data.judge_name_1[index],full_data.judge_name_2[index],full_data.judge_name_3[index]]

    tokens,shape_w2v_gram_features=shape_w2v_gram_features_extraction(text,context_len)
    if index<79:
        _,y_label=tf_training_set_labels_extraction(text,judge_names)
    else:
        _,y_label=tf_test_set_labels_extraction(text)

#     print np.asarray(tokens)[y_label!=0]
#     print np.asarray(y_label)[y_label!=0]
    data_filenames.append(full_data.loc[index,"filename"])
    data_tokens.append(tokens)
    data_features.append(shape_w2v_gram_features)
    data_labels.append(y_label)


[u'Mariya', u'KOLEVA']


In [71]:
data_features=np.asarray(data_features)
data_labels=np.asarray(data_labels)
data_tokens=np.asarray(data_tokens)

In [12]:
num_labels = 4
def reformat(labels):
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return labels


In [13]:
train_dataset=np.vstack(data_features[0:40])
train_dataset=train_dataset.astype(np.float32)
train_labels=reformat(np.hstack(data_labels[0:40]))
train_tokens=np.hstack(data_tokens[0:40])

valid_dataset=np.vstack(data_features[40:55])
valid_dataset=valid_dataset.astype(np.float32)
valid_labels=reformat(np.hstack(data_labels[40:55]))
valid_tokens=np.hstack(data_tokens[40:55])

test_dataset=np.vstack(data_features[55:79])
test_dataset=test_dataset.astype(np.float32)
test_labels=reformat(np.hstack(data_labels[55:79]))
test_tokens=np.hstack(data_tokens[55:79])

#print np.sum(test_labels,0).astype("int")

In [14]:
import tensorflow as tf

In [15]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [30]:
batch_size=256
feature_size=train_dataset.shape[1]
num_labels=train_labels.shape[1]

graph1=tf.Graph()
with graph1.as_default():
    
    tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,feature_size))
    tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    
    tf_valid_dataset=tf.constant(valid_dataset)
    
    tf_test_dataset=tf.placeholder(tf.float32,shape=(None,feature_size))
    
    # Variables.
    weights1 = tf.Variable(tf.truncated_normal([feature_size , int(feature_size*0.7) ]))
    biases1 = tf.Variable(tf.zeros([int(feature_size*0.7) ]))
    weights2 = tf.Variable(tf.truncated_normal([int(feature_size*0.7) , int(feature_size*0.5) ]))
    biases2 = tf.Variable(tf.zeros([int(feature_size*0.5) ]))     
    weights3 = tf.Variable(tf.truncated_normal([int(feature_size*0.5) , num_labels]))
    biases3 = tf.Variable(tf.zeros([num_labels])) 



    # Training computation.
    logits1 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(tf_train_dataset, weights1) + biases1),keep_prob=0.9)
    logits2 =  tf.nn.dropout(tf.nn.relu(tf.matmul(logits1,weights2)+biases2),keep_prob=0.9)
    logits =  tf.matmul(logits2,weights3)+biases3  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    train_prediction = tf.nn.softmax(logits)
    
    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2),weights3)+biases3)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2),weights3)+biases3)    

In [31]:
num_steps = 3001

with tf.Session(graph=graph1) as session1:
    
    tf.initialize_all_variables().run()
    print("Initialized all the variables")

    for step in range(num_steps):
        offset=(step*batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data=train_dataset[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]
        
        feed_dict={tf_train_dataset:batch_data,tf_train_labels:batch_labels}
        _,l,predictions=session1.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            
    weights_saved=[weights1.eval(),weights2.eval(),weights3.eval(),biases1.eval(),biases2.eval(),biases3.eval()]
[nn1_weights1,nn1_weights2,nn1_weights3,nn1_biases1,nn1_biases2,nn1_biases3]=weights_saved

Initialized all the variables
Minibatch loss at step 0: 445.999939
Minibatch accuracy: 3.5%
Validation accuracy: 99.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 99.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%


In [32]:
graph2=tf.Graph()
with graph2.as_default():    
    tf_test_dataset=tf.placeholder(tf.float32,shape=(None,feature_size))
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn1_weights1) + nn1_biases1),nn1_weights2)+nn1_biases2),nn1_weights3)+nn1_biases3)    

In [33]:
with tf.Session(graph=graph2) as session2:
    tf.initialize_all_variables().run()
    feed_dict={tf_test_dataset:train_dataset}
    nn1_train_predictions=session2.run([test_prediction],feed_dict=feed_dict)[0]
    feed_dict={tf_test_dataset:valid_dataset}
    nn1_valid_predictions=session2.run([test_prediction],feed_dict=feed_dict)[0]
    feed_dict={tf_test_dataset:test_dataset}
    nn1_test_predictions=session2.run([test_prediction],feed_dict=feed_dict)[0]

In [34]:
print np.sum(np.argmax(nn1_test_predictions[nn1_test_predictions[:,0]!=1],1)==np.argmax(test_labels[nn1_test_predictions[:,0]!=1],1))
print len(nn1_test_predictions[nn1_test_predictions[:,0]!=1])
print len(test_labels[test_labels[:,0]!=1])

121
145
161


In [35]:
#print test_dataset.shape, nn1_test_predictions.shape, test_labels.shape, len(test_tokens)
for x,y,z in zip(test_tokens[nn1_test_predictions[:,0]!=1],np.argmax(nn1_test_predictions[nn1_test_predictions[:,0]!=1],1),np.argmax(test_labels[nn1_test_predictions[:,0]!=1],1)):
    print x,y,z

Ardiel 1 1
CABRERA 1 1
RUIZ 2 1
Lina 2 2
LAPINSKAITE 2 2
Justyna 3 3
GBYL 3 3
Isabel 1 1
De 1 1
ALFONSETI 2 1
HARTMANN 3 1
Julio 2 2
LAPORTA 3 2
INSA 3 2
Mira 2 3
RAJH 2 3
Andras 1 2
SZASZ 1 2
Antoneta 3 3
CVETIC 3 3
Daniel 1 1
Meredydd 1 1
Anthony 2 1
Ralph 2 2
Pethke 2 2
Sigrid 3 3
Dickmanns 3 3
Kaidi 1 1
TOOMUS 1 1
Lina 2 2
LAPINSKAITE 2 2
Margarida 3 3
CHAMBEL 3 3
GONÇALVES 3 3
Justas 1 1
IVANAUSKAS 1 1
Birgit 2 2
Holst 2 2
FILTENBORG 2 2
Silvie 0 3
4 3 0
Richard 1 1
THEWLIS 1 1
William 2 2
TROTT 3 2
Helena 3 3
NOKKANEN 3 3
Erkki 1 1
MÜNTER 1 1
Ricardo 2 2
NUNES 2 2
FERREIRA 2 2
Natascha 3 3
SEMJEVSKI 3 3
Alain 1 1
RASSAT 1 1
Vanessa 2 2
Patricia 3 3
DE 3 3
CORRES 3 3
Agnes 1 1
SZANYI 1 1
Richard 2 2
BIANCHI 2 2
Agueda 2 3
Las 0 2
Kapnopoulou 3 3
Mauro 1 1
BUFFOLO 1 1
Ralph 2 2
PETHKE 2 2
Anna 3 3
POLITI 3 3
Óscar 1 1
Mondéjar 1 1
Deirdre 2 2
Quinn 3 2
Zoltán 3 3
Szarka 3 3
Birgit 1 1
FILTENBORG 1 1
Raluca 2 2
ARDELEANU 2 2
Paul 3 3
BULLOCK 3 3
Luca 1 1
RAMPINI 1 1
Mira 2 2
RAJH 2 

In [36]:
for index in range(70,79):
    print index
    single_test_dataset=data_features[index].astype("float32")
    single_test_labels=data_labels[index]
    single_test_tokens=np.asarray(data_tokens[index])

    with tf.Session(graph=graph2) as session2:
        tf.initialize_all_variables().run()  
        feed_dict={tf_test_dataset:single_test_dataset}
        nn1_single_test_predictions=session2.run([test_prediction],feed_dict=feed_dict)[0]

    print single_test_tokens[nn1_single_test_predictions[:,0]!=1]
    #print single_test_tokens[np.asarray(single_test_labels)!=0]
    print np.argmax(nn1_single_test_predictions[nn1_single_test_predictions[:,0]!=1],1)
    #print single_test_labels[nn1_single_test_predictions[:,0]!=1]
    print " "

70
[u'Julio' u'LAPORTA' u'INSA' u'Nicole' u'CLARKE']
[1 1 2 2 2]
 
71
[u'Mariya' u'KOLEVA']
[1 3]
 
72
[u'da' u'Liliya' u'YORDANOVA' u'Astrid' u'GRAUL']
[2 1 1 2 3]
 
73
[u'Richard' u'THEWLIS' u'Robert' u'KLIJN' u'BRINKEMA' u'Daniel' u'GAJA']
[1 1 2 2 3 3 3]
 
74
[u'Judit' u'N\xc9METH' u'Alexander' u'SCHIFKO' u'Juan' u'Antonio'
 u'MORALES' u'PAREDES']
[1 1 2 2 2 2 3 3]
 
75
[u'Marc' u'BURKI' u'Fr\xe9d\xe9rique' u'SULPICE' u'Vanessa' u'PAGE']
[1 1 2 2 3 3]
 
76
[u'Birgit' u'FILTENBORG' u'Chantal' u'VAN' u'RIEL' u'Louise' u'THORNING']
[1 1 2 2 2 3 3]
 
77
[u'Fr\xe9d\xe9rique' u'Sulpice' u'Anne-Lee' u'Kristensen' u'Holger' u'Kunz']
[1 1 2 2 3 3]
 
78
[u'Ewa' u'MISZCZAK' u'Andrea' u'NEWMAN' u'Alexandre' u'BERSET']
[1 1 2 2 3 3]
 


In [38]:
context_len_nn_2=6
def nn2_input_feature_extraction(nn1_predictions,context_len_nn_2,num_labels=4):
    temp=np.vstack((np.zeros((context_len_nn_2,num_labels)),nn1_predictions))
    temp2=np.vstack((temp,np.zeros((context_len_nn_2,num_labels))))
    temp2_ngrams=np.asarray(list(ngrams(temp2,context_len_nn_2*2+1)))
    temp2_ngrams=temp2_ngrams.astype("float32")
    return temp2_ngrams.reshape(len(temp2_ngrams),-1)
    

In [39]:
#np.hstack((nn1_train_predictions_grams,train_dataset))

In [40]:
nn1_train_predictions_grams=nn2_input_feature_extraction(nn1_train_predictions,context_len_nn_2,num_labels)
nn1_train_predictions_grams=np.hstack((nn1_train_predictions_grams,train_dataset))
nn1_valid_predictions_grams=nn2_input_feature_extraction(nn1_valid_predictions,context_len_nn_2,num_labels)
nn1_valid_predictions_grams=np.hstack((nn1_valid_predictions_grams,valid_dataset))
nn1_test_predictions_grams=nn2_input_feature_extraction(nn1_test_predictions,context_len_nn_2,num_labels)
nn1_test_predictions_grams=np.hstack((nn1_test_predictions_grams,test_dataset))

In [48]:
batch_size=256
feature_size=nn1_train_predictions_grams.shape[1]
num_labels=train_labels.shape[1]

graph3=tf.Graph()
with graph3.as_default():
    
    nn2_tf_train_dataset=tf.placeholder(tf.float32,shape=(batch_size,feature_size))
    nn2_tf_train_labels=tf.placeholder(tf.float32,shape=(batch_size,num_labels))
    nn2_tf_valid_dataset=tf.constant(nn1_valid_predictions_grams)
    
    
    
    # Variables.
    nn2_weights1 = tf.Variable(tf.truncated_normal([feature_size , int(feature_size*0.7) ]))
    nn2_biases1 = tf.Variable(tf.zeros([int(feature_size*0.7) ]))
    
    nn2_weights2 = tf.Variable(tf.truncated_normal([int(feature_size*0.7) , int(feature_size*0.5) ]))
    nn2_biases2 = tf.Variable(tf.zeros([int(feature_size*0.5) ]))
   
    nn2_weights3 = tf.Variable(tf.truncated_normal([int(feature_size*0.5) , num_labels]))
    nn2_biases3 = tf.Variable(tf.zeros([num_labels])) 



    # Training computation.
    nn2_logits1 = tf.nn.dropout(tf.nn.sigmoid(tf.matmul(nn2_tf_train_dataset, nn2_weights1) + nn2_biases1),keep_prob=0.9)
    nn2_logits2 = tf.nn.dropout(tf.nn.relu(tf.matmul(nn2_logits1,nn2_weights2)+nn2_biases2),keep_prob=0.9)
    nn2_logits =  tf.matmul(nn2_logits2,nn2_weights3)+nn2_biases3
    nn2_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(nn2_logits, nn2_tf_train_labels))
    nn2_train_prediction = tf.nn.softmax(nn2_logits)
    
    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(0.25).minimize(nn2_loss)

    # Predictions for the training, validation, and test data.
    # Predictions for the training, validation, and test data.
    #valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(nn2_tf_valid_dataset, nn2_weights1) + nn2_biases1),nn2_weights2)+nn2_biases2)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(nn2_tf_valid_dataset, nn2_weights1) + nn2_biases1),nn2_weights2)+nn2_biases2),nn2_weights3)+nn2_biases3)    
    #test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(nn2_tf_test_dataset, nn2_weights1) + nn2_biases1),nn2_weights2)+nn2_biases2),nn2_weights3)+nn2_biases3)    
    
    

In [49]:
num_steps = 3001

with tf.Session(graph=graph3) as session3:
    
    tf.initialize_all_variables().run()
    print("Initialized all the variables")
    
    #for num in range(3):
    for step in range(num_steps):
        offset=(step*batch_size) % (train_labels.shape[0] - batch_size)

        batch_data=nn1_train_predictions_grams[offset:(offset+batch_size),:]
        batch_labels=train_labels[offset:(offset+batch_size),:]


        feed_dict={nn2_tf_train_dataset:batch_data,nn2_tf_train_labels:batch_labels}
        _,l,predictions=session3.run([optimizer, nn2_loss, nn2_train_prediction], feed_dict=feed_dict)

        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
            
    weights_saved=[nn2_weights1.eval(),nn2_weights2.eval(),nn2_weights3.eval(),nn2_biases1.eval(),nn2_biases2.eval(),nn2_biases3.eval()]
    #weights_saved=[nn2_weights1.eval(),nn2_weights2.eval(),nn2_biases1.eval(),nn2_biases2.eval()]
#[nn2_weights1_saved,nn2_weights2_saved,nn2_biases1_saved,nn2_biases2_saved]=weights_saved
[nn2_weights1_saved,nn2_weights2_saved,nn2_weights3_saved,nn2_biases1_saved,nn2_biases2_saved,nn2_biases3_saved]=weights_saved

Initialized all the variables
Minibatch loss at step 0: 13.444063
Minibatch accuracy: 88.7%
Validation accuracy: 99.8%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 100.0%


In [50]:
graph4=tf.Graph()
with graph4.as_default():    
    tf_test_dataset=tf.placeholder(tf.float32,shape=(None,feature_size))
    #test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved),nn2_weights3_saved)+nn2_biases3_saved)    

In [51]:
with tf.Session(graph=graph4) as session4:
    tf.initialize_all_variables().run()
    feed_dict={tf_test_dataset:nn1_train_predictions_grams}
    nn2_train_predictions=session4.run([test_prediction],feed_dict=feed_dict)[0]
    feed_dict={tf_test_dataset:nn1_valid_predictions_grams}
    nn2_valid_predictions=session4.run([test_prediction],feed_dict=feed_dict)[0]
    feed_dict={tf_test_dataset:nn1_test_predictions_grams}
    nn2_test_predictions=session4.run([test_prediction],feed_dict=feed_dict)[0]

In [53]:
print np.sum(np.argmax(nn2_test_predictions[nn2_test_predictions[:,0]<0.5],1)==np.argmax(test_labels[nn2_test_predictions[:,0]<0.5],1))
print len(nn2_test_predictions[nn1_test_predictions[:,0]<0.5])
print len(test_labels[test_labels[:,0]!=1])

123
143
161


In [100]:
for index in range(80,100):
    print(" ")
    print index
    single_test_dataset=data_features[index].astype("float32")
    single_test_labels=data_labels[index]
    single_test_tokens=np.asarray(data_tokens[index])
    
    graph5=tf.Graph()
    with graph5.as_default():    
        tf_test_dataset=tf.placeholder(tf.float32,shape=(None,None))
        test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn1_weights1) + nn1_biases1),nn1_weights2)+nn1_biases2),nn1_weights3)+nn1_biases3)    
        
    with tf.Session(graph=graph5) as session5:
        tf.initialize_all_variables().run()  
        feed_dict={tf_test_dataset:single_test_dataset}
        nn1_single_test_predictions=session5.run([test_prediction],feed_dict=feed_dict)[0]
        
    nn1_single_test_predictions_grams=nn2_input_feature_extraction(nn1_single_test_predictions,context_len_nn_2,num_labels)
    nn1_single_test_predictions_grams=np.hstack((nn1_single_test_predictions_grams,single_test_dataset))
    
    graph6=tf.Graph()
    with graph6.as_default():    
        tf_test_dataset=tf.placeholder(tf.float32,shape=(None,None))
        #test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved)
        test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved),nn2_weights3_saved)+nn2_biases3_saved)    
        
    with tf.Session(graph=graph6) as session6:
        tf.initialize_all_variables().run()
        feed_dict={tf_test_dataset:nn1_single_test_predictions_grams}
        nn2_single_test_predictions=session6.run([test_prediction],feed_dict=feed_dict)[0]
    
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,1]==1])
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,2]==1])
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,3]==1])
    print(" ")
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,1]==1])
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,2]==1])
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,3]==1])

 

 
80
Loreto URRACA
LUQUE Ana PAREJA
Paul BULLOCK
 
Loreto URRACA
LUQUE Ana PAREJA
Paul BULLOCK
 
81
Marcela Langerová
Ricardo Nunes Ferreira
Janja Felc
 
Marcela Langerová
Ricardo Nunes
Ferreira Janja Felc
 
82
David LEFFLER
Andrea NEWMAN
Raoul ERARD
 
David LEFFLER
Andrea
Raoul ERARD
 
83
David LEFFLER
Hugh O’NEILL
Lucie TRPÍKOVÁ
 
David LEFFLER
Hugh O’NEILL
Lucie TRPÍKOVÁ
 
84
Ardiel CABRERA
RUIZ Kaidi TOOMUS
Julio LAPORTA
 
Ardiel CABRERA
RUIZ Kaidi
TOOMUS Julio LAPORTA
 
85
Deirdre QUINN
Sandrine
Paul BULLOCK
 
Deirdre QUINN
Sandrine BUORO
Paul BULLOCK
 
86
dissect Swetlana BRAUN
Ewelina

 
Swetlana BRAUN
Ewelina ŚLIWIŃSKA
Silvie
 
87
Daniel GÁJA
Mira
RAJH Radka STUPKOVÁ
 
Daniel GÁJA
Mira
Radka STUPKOVÁ
 
88
Paul BULLOCK
Inés GARCÍA
LLEDÓ Margarida CHAMBEL
 
Paul BULLOCK
Inés GARCÍA
LLEDÓ Margarida CHAMBEL
 
89
lsabel DE
ALFONSETI URRACA LUQUE Alvaro
Loreto SESMA MERINO
 
lsabel DE ALFONSETI
URRACA LUQUE
Alvaro SESMA MERINO
 
90
Patricia LOPEZ
FERNANDEZ CORRES Holger
Vanessa PAGE


In [76]:
print len(single_test_tokens)
print single_test_tokens[nn1_single_test_predictions[:,0]!=1]
print single_test_tokens[nn2_single_test_predictions[:,0]!=1]

4717
[u'Ardiel' u'CABRERA' u'RUIZ' u'Kaidi' u'TOOMUS' u'Julio' u'LAPORTA']
[u'Ardiel' u'CABRERA' u'RUIZ' u'Kaidi' u'TOOMUS' u'Julio' u'LAPORTA']


In [109]:
predictions_1_db=full_data.copy(deep=True)
predictions_2_db=full_data.copy(deep=True)

In [133]:
for index in range(80,200):
    print(" ")
    print index
    single_test_dataset=data_features[index].astype("float32")
    single_test_labels=data_labels[index]
    single_test_tokens=np.asarray(data_tokens[index])
    
    graph5=tf.Graph()
    with graph5.as_default():    
        tf_test_dataset=tf.placeholder(tf.float32,shape=(None,None))
        test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn1_weights1) + nn1_biases1),nn1_weights2)+nn1_biases2),nn1_weights3)+nn1_biases3)    
        
    with tf.Session(graph=graph5) as session5:
        tf.initialize_all_variables().run()  
        feed_dict={tf_test_dataset:single_test_dataset}
        nn1_single_test_predictions=session5.run([test_prediction],feed_dict=feed_dict)[0]
        
    nn1_single_test_predictions_grams=nn2_input_feature_extraction(nn1_single_test_predictions,context_len_nn_2,num_labels)
    nn1_single_test_predictions_grams=np.hstack((nn1_single_test_predictions_grams,single_test_dataset))
    
    graph6=tf.Graph()
    with graph6.as_default():    
        tf_test_dataset=tf.placeholder(tf.float32,shape=(None,None))
        #test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved)
        test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.sigmoid(tf.matmul(tf_test_dataset, nn2_weights1_saved) + nn2_biases1_saved),nn2_weights2_saved)+nn2_biases2_saved),nn2_weights3_saved)+nn2_biases3_saved)    
        
    with tf.Session(graph=graph6) as session6:
        tf.initialize_all_variables().run()
        feed_dict={tf_test_dataset:nn1_single_test_predictions_grams}
        nn2_single_test_predictions=session6.run([test_prediction],feed_dict=feed_dict)[0]
    
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,1]==1])
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,2]==1])
    print ' '.join(single_test_tokens[nn1_single_test_predictions[:,3]==1])
    print(" ")
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,1]==1])
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,2]==1])
    print ' '.join(single_test_tokens[nn2_single_test_predictions[:,3]==1])

    predictions_1_db.loc[index,"judge_name_1"]= ' '.join(single_test_tokens[nn1_single_test_predictions[:,1]==1])
    predictions_1_db.loc[index,"judge_name_2"]= ' '.join(single_test_tokens[nn1_single_test_predictions[:,2]==1])
    predictions_1_db.loc[index,"judge_name_3"]= ' '.join(single_test_tokens[nn1_single_test_predictions[:,3]==1])
    print(" ")
    predictions_2_db.loc[index,"judge_name_1"]= ' '.join(single_test_tokens[nn2_single_test_predictions[:,1]==1])
    predictions_2_db.loc[index,"judge_name_2"]= ' '.join(single_test_tokens[nn2_single_test_predictions[:,2]==1])
    predictions_2_db.loc[index,"judge_name_3"]= ' '.join(single_test_tokens[nn2_single_test_predictions[:,3]==1])

    

 
80
Loreto URRACA
LUQUE Ana PAREJA
Paul BULLOCK
 
Loreto URRACA
LUQUE Ana PAREJA
Paul BULLOCK
 
 
81
Marcela Langerová
Ricardo Nunes Ferreira
Janja Felc
 
Marcela Langerová
Ricardo Nunes
Ferreira Janja Felc
 
 
82
David LEFFLER
Andrea NEWMAN
Raoul ERARD
 
David LEFFLER
Andrea
Raoul ERARD
 
 
83
David LEFFLER
Hugh O’NEILL
Lucie TRPÍKOVÁ
 
David LEFFLER
Hugh O’NEILL
Lucie TRPÍKOVÁ
 
 
84
Ardiel CABRERA
RUIZ Kaidi TOOMUS
Julio LAPORTA
 
Ardiel CABRERA
RUIZ Kaidi
TOOMUS Julio LAPORTA
 
 
85
Deirdre QUINN
Sandrine
Paul BULLOCK
 
Deirdre QUINN
Sandrine BUORO
Paul BULLOCK
 
 
86
dissect Swetlana BRAUN
Ewelina

 
Swetlana BRAUN
Ewelina ŚLIWIŃSKA
Silvie
 
 
87
Daniel GÁJA
Mira
RAJH Radka STUPKOVÁ
 
Daniel GÁJA
Mira
Radka STUPKOVÁ
 
 
88
Paul BULLOCK
Inés GARCÍA
LLEDÓ Margarida CHAMBEL
 
Paul BULLOCK
Inés GARCÍA
LLEDÓ Margarida CHAMBEL
 
 
89
lsabel DE
ALFONSETI URRACA LUQUE Alvaro
Loreto SESMA MERINO
 
lsabel DE ALFONSETI
URRACA LUQUE
Alvaro SESMA MERINO
 
 
90
Patricia LOPEZ
FERNANDEZ CORRES 

In [132]:
from pandas import ExcelWriter
writer = ExcelWriter('euipo_english_judges_pred_1.xlsx',encoding='unicode')
predictions_1_db.loc[:,"filename":"judge_name_3"].to_excel(writer,'Sheet1')
writer.save()
writer = ExcelWriter('euipo_english_judges_pred_2.xlsx',encoding='unicode')
predictions_2_db.loc[:,"filename":"judge_name_3"].to_excel(writer,'Sheet2')
writer.save()